In [6]:
import dask as da
import dask.dataframe as daskdf
import dask.array as daa
import dask.distributed as dd
import dask.datasets as ds
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
import sklearn as sk
import numpy as np
import dask_ml.preprocessing as dm_pre
import dask_ml.cluster as dm_cluster
import dask_geopandas as dg
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import geopandas as gpd
from scipy.stats import pearsonr
import time

# Phase Four: Modeling. 
This phase involves selecting, applying, and tuning various statistical or machine learning models to your prepared data. Here's what typically happens in this phase:

In [7]:
# load preprocessed data
"""data look like this: ;mapped_veh_id;timestamps_UTC;lat;lon;RS_E_InAirTemp_PC1;RS_E_InAirTemp_PC2;RS_E_OilPress_PC1;RS_E_OilPress_PC2;RS_E_RPM_PC1;RS_E_RPM_PC2;RS_E_WatTemp_PC1;RS_E_WatTemp_PC2;RS_T_OilTemp_PC1;RS_T_OilTemp_PC2
0;181;2023-08-01 03:44:12;50.7698183;3.8721144;27.0;23.0;255.0;238.0;794.0;801.0;83.0;81.0;76.0;77.0
1;143;2023-08-01 06:36:29;51.0399934;3.6934285;33.0;32.0;272.0;324.0;802.0;804.0;78.0;78.0;73.0;74.0
2;183;2023-08-24 06:53:54;50.7422026;3.6020347;31.0;33.0;234.0;182.0;799.0;802.0;82.0;82.0;85.0;87.0"""
# Load data
def load_data():
    print("Loading data...")
    start = time.time()
    df = daskdf.read_csv("preprocessed.csv")
    end = time.time()
    print("Data loaded in {} seconds".format(end-start))
    return df

In [8]:
ddf = load_data()
print(ddf.head())
print(len(ddf))

Loading data...
Data loaded in 0.020537137985229492 seconds
   Unnamed: 0  mapped_veh_id       timestamps_UTC        lat       lon  \
0           0          112.0  2023-08-01 11:42:55  51.031514  3.727847   
1           1          179.0  2023-08-24 17:48:07  51.189722  5.110089   
2           2          190.0  2023-08-01 12:27:13  51.136007  4.559994   
3           3          179.0  2023-08-24 18:18:05  51.190734  5.114649   
4           4          122.0  2023-08-01 12:53:12  51.035850  3.709883   

   RS_E_InAirTemp_PC1  RS_E_InAirTemp_PC2  RS_E_OilPress_PC1  \
0              309.15              313.15              276.0   
1              314.15              303.15              220.0   
2              307.15              313.15              376.0   
3              311.15              304.15              220.0   
4              303.15              304.15              193.0   

   RS_E_OilPress_PC2  RS_E_RPM_PC1  RS_E_RPM_PC2  RS_E_WatTemp_PC1  \
0              248.0         798.0      

In [9]:
# Modeling techniques selection